##推薦作法:
- 將需要的資料都存成變數：
    - 新聞內容：class(python裡class是保留字，所以我在下面的code改成category),title,newsDate,journalist,content,hit,url,source
    - 評論：url,commenter,comment
- 將這些變數用list或dictionary存起來
- 將存的list或dictionary轉成pandas
- 直接用DataFrame.to_csv(filename,encoding="utf-8")轉成csv
- 下面的code有存進資料庫的部分，如果不存資料庫的人可以把那些註解掉，存成csv就好，如果要存進資料庫，則記得先見好資料庫，資料庫的code詳見news.sql

## 抓取單一日期或日期區間  ETToday_day(startday,endday,category=["政治","財經","國際","大陸"],sleep_time=0) 

In [268]:
# sleep_time意思為抓取每一天新聞之間要休息多久，單位為秒
# year, month要輸入integer(ex. 2012,3)
# 如果抓本月的新聞，會預設抓到今天前一天的新聞

import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import math
import re
from datetime import date,datetime, timedelta 
import time
import pandas as pd
from pandas.io import sql
import traceback
import string  


#################連資料庫用
def connect(dbname,password,host,user,port=3306):
    import MySQLdb
    # 承德電腦 host="10.120.30.1"
    # localhost host = '127.0.0.1'
    db = MySQLdb.connect(host=host,user="root",passwd=password,db=dbname,charset="utf8")
    return db
def insert_news(db,category=None,title=None,newsDate=None,newsTime=None,journalist=None,content=None,hit=None,url=None,source="ETToday"):
    cur = db.cursor()
    cur.execute("insert into news(class,title,newsDate,newsTime,journalist,content,hit,url,source) values(%s,%s,%s,%s,%s,%s,%s,%s,%s)",[category,title,newsDate,newsTime,journalist,content,hit,url,source])
    db.commit()
def insert_comments(db,commentDate,commentTime,commenter,commenterUrl,comment,url,remark=None):
    cur = db.cursor()
    cur.execute("insert into comments(commentDate,commentTime,commenter,commenterUrl,comment,url)values(%s,%s,%s,%s,%s,%s)" ,[commentDate,commentTime,commenter,commenterUrl,comment,url])
    db.commit()
def insert_newsRaw(db,url,newsRaw):
    cur = db.cursor()
    cur.execute("insert into newsRaw(url,newsRaw)values(%s,%s)",[url,newsRaw])
    db.commit()
def close(db):
    db.close()
#################連資料庫用

sentinel = object()
def ETToday_day(startday=(str(date.today()-timedelta(1))),endday=sentinel,category=["政治","財經","國際","大陸"],sleep_time=0):
     
    if endday is sentinel:
        endday=startday
    
    def convert_day(d):
        return datetime.strptime(d, '%Y-%m-%d')
    
    ed=convert_day(endday)
    sd=convert_day(startday)

    assert ed >= sd, "endday must after startday"
    
    ds = []
    for i in xrange(0,(ed-sd).days+1):
        ds.append(str((ed- timedelta(i)).strftime('%Y-%m-%d')))

    url_all =[]
    for newsDate in ds:
        url_dic = {"政治":"http://www.ettoday.net/news/news-list-%s-1.htm"%newsDate,
                    "財經":"http://www.ettoday.net/news/news-list-%s-17.htm"%newsDate,
                    "國際":"http://www.ettoday.net/news/news-list-%s-2.htm"%newsDate,
                    "大陸":"http://www.ettoday.net/news/news-list-%s-3.htm"%newsDate,
                    "社會":"http://www.ettoday.net/news/news-list-%s-6.htm"%newsDate,
                    "地方":"http://www.ettoday.net/news/news-list-%s-7.htm"%newsDate,
                    "影劇":"http://www.ettoday.net/news/news-list-%s-9.htm"%newsDate,
                    "3C":"http://www.ettoday.net/news/news-list-%s-20.htm"%newsDate
                   }
        url_categories = {}
        for cat in category:
            try:
                url_categories[str(cat)]=url_dic[str(cat)]
                url_all.append(url_categories[str(cat)])
            except:
                print newsDate, cat, url_dic[str(cat)], "\n"
                print traceback.format_exc() + "\n"
    
    identify = string.maketrans('', '')
    delEStr = string.punctuation
    L = []
    for cat in url_all:
        try:
            res = requests.get(cat)
            res.encoding="utf-8"
            soup = BeautifulSoup(res.text)
            page = re.search('\((\d+).\)',soup.select(".info")[0].text).group(1)
            pageCount = int(math.ceil(float(page)/30))
            soups = [soup]
        except:
            print cat
            raise 
        if pageCount >1:
            for i in xrange(2,pageCount+1):
                curl = str(cat).split(".htm")[0] + "-" + str(i) + ".htm"
                res = requests.get(curl)
                res.encoding="utf-8"
                soups.append(BeautifulSoup(res.text))

        for soup in soups:         
            for block in soup.select("#all-news-list > h3"):
                try:
                    timeRegex=re.search("\[(\d\d)\/(\d\d) (\d{1,2}\:\d{1,2})\]",block.select('span')[0].text)
                    testDate=timeRegex.group(1)+'-'+timeRegex.group(2)
                    if not testDate == re.search('\d{4}-(\d\d-\d\d)',newsDate).group(1).decode('utf-8'):
                        continue
                    newsTime=timeRegex.group(3)
                    title=block.select('a')[0].text
                    aLink=block.select('a')[0]['href']
                except:
                    print cat
                    raise
                #########get page breakpoint
                #driver = webdriver.Firefox()
                #driver = webdriver.Chrome()
                driver = webdriver.PhantomJS()
                webdriver.DesiredCapabilities.PHANTOMJS['phantomjs.page.customHeaders.Accept'] = 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'
                webdriver.DesiredCapabilities.PHANTOMJS['phantomjs.page.customHeaders.Accept-Encoding'] = 'gzip, deflate, sdch'
                webdriver.DesiredCapabilities.PHANTOMJS['phantomjs.page.customHeaders.Cache-Control'] = 'max-age=0'
                webdriver.DesiredCapabilities.PHANTOMJS['phantomjs.page.customHeaders.Connection'] = 'keep-alive'
                webdriver.DesiredCapabilities.PHANTOMJS['phantomjs.page.customHeaders.User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.124 Safari/537.36'
                try:
                    driver.get(aLink)
                    WebDriverWait(driver, 30).until(
                            EC.visibility_of_element_located((By.ID, "fbComments"))
                        )
                    html = driver.page_source
                    cont = str(html.encode('utf-8'))
                    soupCont = BeautifulSoup(cont)
                except Exception as e:
                    f=open("ETTodayException.log","a")
                    ExList=[
                        "Log at: " + str(datetime.today()) + "\n",
                        "\t -News Date: " + newsDate + "\n",
                        "\t -News Category: " + cat + "\n",
                        "\t -News url: " + aLink + "\n",
                        "\t -BreakPoint: get page" + "\n",
                        "\t -Exception Type: " + str(e.__class__) + "\n",
                        "\t -Exception Message: " + e.message + "\n",
                        "\t -Exception Traceback: \n", 
                        str(traceback.format_exc()) + "\n"
                    ]
                    f.writelines(ExList)
                    f.close
                    continue
                finally:
                    driver.quit()
                #########get page breakpoint

                #########content breakpoint
                try:
                    content=""
                    contentRaw = soupCont.select('.story > sectione > p')
                    try:
                        for i in xrange(0,len(contentRaw)):
                            for con in contentRaw[i].select('img'):
                                contentRaw[i].img.decompose()
                    except:
                        pass
                    for p in contentRaw:
                        content = content + p.text
                    content=content.strip()
                except Exception as e:
                    f=open("ETTodayException.log","a")
                    ExList=[
                        "Log at: " + str(datetime.today()) + "\n",
                        "\t -News Date: " + newsDate + "\n",
                        "\t -News Category: " + cat + "\n",
                        "\t -News url: " + aLink + "\n",
                        "\t -BreakPoint: content" + "\n",
                        "\t -Exception Type: " + str(e.__class__) + "\n",
                        "\t -Exception Message: " + e.message + "\n",
                        "\t -Exception Traceback: \n", 
                        str(traceback.format_exc()) + "\n"
                    ]
                    f.writelines(ExList)
                    f.close
                #########content breakpoint

                #########journist breakpoint
                jor = str(BeautifulSoup(cont).select('.story')[0].prettify().encode("utf-8"))
                try:
                    journalist=re.search('([^圖].*?)／.*?報導.*?', jor).group(1).strip()
                    journalist=journalist.translate(identify, delEStr)
                    if '記者' in journalist:
                        journalist=re.sub('記者','',journalist)
                    journalist=re.sub('、', '', journalist)
                except:
                    try:
                        journalist=re.search('文／(.*?)<',cont).group(1).strip()
                        journalist=journalist.translate(identify, delEStr)
                        journalist=re.sub('、', '', journalist)
                    except Exception as e:
                        try:
                            journalist=None
                            f=open("ETTodayException.log","a")
                            ExList=[
                                "Log at: " + str(datetime.today()) + "\n",
                                "\t -News Date: " + newsDate + "\n",
                                "\t -News Category: " + cat + "\n",
                                "\t -News url: " + aLink + "\n",
                                "\t -BreakPoint: journist" + "\n",
                                "\t -Exception Type: " + str(e.__class__) + "\n",
                                "\t -Exception Message: " + e.message + "\n",
                                "\t -Exception Traceback: \n ", 
                                str(traceback.format_exc().decode('utf-8').encode('utf-8')) + "\n"
                            ]
                            f.writelines(ExList)
                            f.close
                        except:
                            print "jor: ",aLink," WTF"
                #########journist breakpoint

                #########category breakpoint
                category=block.select('em')[0].text
                '''
                if not category == cat.decode("utf-8"):
                    try:
                        raise Exception
                    except Exception as e:
                        f=open("ETTodayException.log","a")
                        ExList=[
                            "Log at: " + str(datetime.today()) + "\n",
                            "\t -News Date: " + newsDate + "\n",
                            "\t -News Category: " + cat + "\n",
                            "\t -News url: " + aLink + "\n",
                            "\t -BreakPoint: category" + "\n",
                            "\t -Exception Type: " + str(e.__class__) + "\n",
                            "\t -Exception Message: " + e.message + "\n",
                            "\t -Exception Traceback: \n", 
                            str(traceback.format_exc()) + "\n"
                        ]
                        f.writelines(ExList)
                        f.close
                '''
                #########category breakpoint

                try:
                    L.append({'class':category,'title':title,'newsDate':newsDate,'newsTime':newsTime,'journalist':journalist,'content':content,'hit':hit,'url':aLink,'source':'ETToday'})
                except:
                    pass

                #########comment breakpoint
                comments=[]
                try:
                    commLink0 = soupCont.find("div",re.compile('fb-comments.*'))
                    commLink1= commLink0.select(' span > iframe')[0]['src']
                    commLink=re.sub(r"numposts=\d{1,3}&", "numposts=100&", commLink1)
                    resComm = requests.get(commLink)
                    resComm.encoding = "utf-8"
                    soupComm = BeautifulSoup(resComm.text)      
                    commentBlock=soupComm.select('.postContainer.fsl.fwb.fcb')
                    for comm in commentBlock: 
                        dt=re.search("(\d\d\d\d).(\d{1,2}).(\d{1,2}). (\d{1,2}\:\d{1,2})",comm.select('abbr')[0]['title'])
                        commentDate="%s-%02d-%02d"%(dt.group(1),int(dt.group(2)),int(dt.group(3)))
                        commentTime=dt.group(4)
                        commenter=comm.select('.profileName')[0].text
                        commenterUrl=comm.select('.profileName')[0]['href']
                        comment=comm.select('.postText')[0].text
                        comments.append({'commentDate':commentDate,'commentTime':commentTime,'commenter':commenter,'commenterUrl':commenterUrl,'comment':comment,'url':aLink,'remark':None})
                except Exception as e:
                    f=open("ETTodayException.log","a")
                    ExList=[
                        "Log at: " + str(datetime.today()) + "\n",
                        "\t -News Date: " + newsDate + "\n",
                        "\t -News Category: " + cat + "\n",
                        "\t -News url: " + aLink + "\n",
                        "\t -BreakPoint: comment" + "\n",
                        "\t -Exception Type: " + str(e.__class__) + "\n",
                        "\t -Exception Message: " + e.message + "\n",
                        "\t -Exception Traceback: \n", 
                        str(traceback.format_exc()) + "\n"
                    ]
                    f.writelines(ExList)
                    f.close
                #########comment breakpoint

                #################連資料庫用
                hit = None
                #print category,title,newsDate,newsTime,journalist,content,hit,aLink,commentDate,commentTime,commenter,comment
                try:
                    conn=connect("practice","oblivn0374","localhost","root")
                    insert_news(conn,category,title,newsDate,newsTime,journalist,content,hit,aLink)
                    insert_newsRaw(conn,aLink,res.text)
                    for c in comments:
                        insert_comments(conn,c.values()[3],c.values()[6],c.values()[4],c.values()[1],c.values()[0],c.values()[5])
                except:
                    print "something not write in DB \n", traceback.format_exc(),aLink,"\n"
                finally:
                    close(conn)
                #################連資料庫用
                time.sleep(sleep_time)

    
    #########write_in_csv breakpoint
    try:
        # pandas - content
        df = pd.DataFrame(L,columns =['class','title','newsDate','newsTime','journalist','content','hit','url','source'])
        # pandas - comment
        df_comm = pd.DataFrame(comments,columns =['commentDate','commentTime','commenter','commenterUrl','comment','url','remark'])
        
        # write csv
        if startday==endday:
            std="".join(startday.split('-'))
            filename_content="ETTdoay_content_"+std+".csv"
            filename_comment="ETTdoay_comment_"+std+".csv"
        else:
            std="".join(startday.split('-'))
            end="".join(endday.split('-'))
            filename_content="ETTdoay_content_"+"_"+std+"-"+end+".csv"
            filename_comment="ETTdoay_comment_"+"_"+std+"-"+end+".csv"
        ## write content in csv
        f=open(filename_content,"w")
        df.to_csv(filename_content,encoding="utf-8")
        f.close()
        ## write comment in csv
        f=open(filename_comment,"w")
        df_comm.to_csv(filename_comment,encoding="utf-8")
        f.close()
        
    except Exception as e:
        f=open("ETTodayException.log","a")
        ExList=[
            "Log at: " + str(datetime.today()) + "\n",
            "\t -News Date: " + newsDate + "\n",
            "\t -News Category: " + cat + "\n",
            "\t -News url: " + aLink + "\n",
            "\t -BreakPoint: write_in_csv" + "\n",
            "\t -Exception Type: " + str(e.__class__) + "\n",
            "\t -Exception Message: " + e.message + "\n",
            "\t -Exception Traceback: \n", 
            str(traceback.format_exc()) + "\n"
        ]
        f.writelines(ExList)
        f.close
#########write_in_csv breakpoint

In [269]:
# run
a = time.clock()
ETToday_day('2015-05-02',category=["政治","財經","國際","大陸"])
time.clock() - a 

jor:  http://www.ettoday.net/news/20150502/499637.htm  WTF
jor:  http://www.ettoday.net/news/20150502/499555.htm  WTF


KeyboardInterrupt: 

## 抓取單一或多月份  ETToday_month(year,month,year2,month2,category=["政治","財經","國際","大陸"],sleep_time=0) 

In [ ]:
# sleep_time意思為抓取每一天新聞之間要休息多久，單位為秒
# year, month要輸入integer(ex. 2012,3)
# 如果抓本月的新聞，會預設抓到今天前一天的新聞

import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import math
import re
from datetime import date,datetime, timedelta 
import time
import pandas as pd
from pandas.io import sql
import traceback
import string

#################連資料庫用
def connect(dbname,password,host,user,port=3306):
    import MySQLdb
    # 承德電腦 host="10.120.30.1"
    # localhost host = '127.0.0.1'
    db = MySQLdb.connect(host=host,user="root",passwd=password,db=dbname,charset="utf8")
    return db
def insert_news(db,category=None,title=None,newsDate=None,newsTime=None,journalist=None,content=None,hit=None,url=None,source="ETToday"):
    cur = db.cursor()
    cur.execute("insert into news(class,title,newsDate,newsTime,journalist,content,hit,url,source) values(%s,%s,%s,%s,%s,%s,%s,%s,%s)",[category,title,newsDate,newsTime,journalist,content,hit,url,source])
    db.commit()
def insert_comments(db,commentDate,commentTime,commenter,commenterUrl,comment,url,remark=None):
    cur = db.cursor()
    cur.execute("insert into comments(commentDate,commentTime,commenter,commenterUrl,comment,url)values(%s,%s,%s,%s,%s,%s)" ,[commentDate,commentTime,commenter,commenterUrl,comment,url])
    db.commit()
def insert_newsRaw(db,url,newsRaw):
    cur = db.cursor()
    cur.execute("insert into newsRaw(url,newsRaw)values(%s,%s)",[url,newsRaw])
    db.commit()
def close(db):
    db.close()
#################連資料庫用

sentinel = object()
def ETToday_month(year,month,year2=sentinel,month2=sentinel,category=["政治","財經","國際","大陸"],sleep_time=0):
    
    if year2 is sentinel:
        year2 = year
    if month2 is sentinel:
        month2 = month
    
    assert date(year2,month2,1) >= date(year,month,1), "year2,month2 must after year,month"

    def first_day_of_month_d(d):
        return date(d.year, d.month, 1)

    def first_day_of_month(y,m):
        return date(y,m,1)

    def last_day_of_month(y,m):
        return date(y,m+1,1) - timedelta(days = 1)

    def diff_months_d(d1, d2):
        return (d1.year - d2.year)*12 + d1.month - d2.month

    def diff_days(y,m,y2,m2):
        return abs(last_day_of_month(y2,m2)-first_day_of_month(y,m)).days

    # check if input month exceed current month
    assert diff_months_d(date.today(),first_day_of_month(year,month))>=0, "眼睛睜大點好嘛？%s月還沒到ok？"%month
    assert diff_months_d(date.today(),first_day_of_month(year2,month2))>=0, "眼睛睜大點好嘛？%s月還沒到ok？"%month2

    if diff_months_d(date.today(),first_day_of_month(year2,month2))==0:
        duration=(date.today()-first_day_of_month_d(date.today())).days
        ds = []
        for i in xrange(1,duration+1):
            ds.append(str((date.today()- timedelta(i)).strftime('%Y-%m-%d')))
    else:
        #duration=calendar.monthrange(year,month)[1]
        duration=diff_days(year,month,year2,month2)
        ds = []
        for i in xrange(0,duration+1):
            ds.append(str((last_day_of_month(year2,month2)- timedelta(i)).strftime('%Y-%m-%d')))
    
    url_all =[]
    for newsDate in ds:
        url_dic = {"政治":"http://www.ettoday.net/news/news-list-%s-1.htm"%newsDate,
                    "財經":"http://www.ettoday.net/news/news-list-%s-17.htm"%newsDate,
                    "國際":"http://www.ettoday.net/news/news-list-%s-2.htm"%newsDate,
                    "大陸":"http://www.ettoday.net/news/news-list-%s-3.htm"%newsDate,
                    "社會":"http://www.ettoday.net/news/news-list-%s-6.htm"%newsDate,
                    "地方":"http://www.ettoday.net/news/news-list-%s-7.htm"%newsDate,
                    "影劇":"http://www.ettoday.net/news/news-list-%s-9.htm"%newsDate,
                    "3C":"http://www.ettoday.net/news/news-list-%s-20.htm"%newsDate
                   }
        url_categories = {}
        for cat in category:
            try:
                url_categories[str(cat)]=url_dic[str(cat)]
                url_all.append(url_categories[str(cat)])
            except:
                print newsDate, cat, url_dic[str(cat)], "\n"
                print traceback.format_exc() + "\n"

    identify = string.maketrans('', '')
    delEStr = string.punctuation
    L = []
    for cat in url_all:
        try:
            res = requests.get(cat)
            res.encoding="utf-8"
            soup = BeautifulSoup(res.text)
            page = re.search('\((\d+).\)',soup.select(".info")[0].text).group(1)
            pageCount = int(math.ceil(float(page)/30))
            soups = [soup]
        except:
            print cat
            raise 
        if pageCount >1:
            for i in xrange(2,pageCount+1):
                curl = str(cat).split(".htm")[0] + "-" + str(i) + ".htm"
                res = requests.get(curl)
                res.encoding="utf-8"
                soups.append(BeautifulSoup(res.text))

        for soup in soups:         
            for block in soup.select("#all-news-list > h3"):
                try:
                    timeRegex=re.search("\[(\d\d)\/(\d\d) (\d{1,2}\:\d{1,2})\]",block.select('span')[0].text)
                    testDate=timeRegex.group(1)+'-'+timeRegex.group(2)
                    if not testDate == re.search('\d{4}-(\d\d-\d\d)',newsDate).group(1).decode('utf-8'):
                        continue
                    newsTime=timeRegex.group(3)
                    title=block.select('a')[0].text
                    aLink=block.select('a')[0]['href']
                except:
                    print cat
                    raise
                #########get page breakpoint
                #driver = webdriver.Firefox()
                #driver = webdriver.Chrome()
                driver = webdriver.PhantomJS()
                webdriver.DesiredCapabilities.PHANTOMJS['phantomjs.page.customHeaders.Accept'] = 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'
                webdriver.DesiredCapabilities.PHANTOMJS['phantomjs.page.customHeaders.Accept-Encoding'] = 'gzip, deflate, sdch'
                webdriver.DesiredCapabilities.PHANTOMJS['phantomjs.page.customHeaders.Cache-Control'] = 'max-age=0'
                webdriver.DesiredCapabilities.PHANTOMJS['phantomjs.page.customHeaders.Connection'] = 'keep-alive'
                webdriver.DesiredCapabilities.PHANTOMJS['phantomjs.page.customHeaders.User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.124 Safari/537.36'
                try:
                    driver.get(aLink)
                    WebDriverWait(driver, 30).until(
                            EC.visibility_of_element_located((By.ID, "fbComments"))
                        )
                    html = driver.page_source
                    cont = str(html.encode('utf-8'))
                    soupCont = BeautifulSoup(cont)
                except Exception as e:
                    f=open("ETTodayException.log","a")
                    ExList=[
                        "Log at: " + str(datetime.today()) + "\n",
                        "\t -News Date: " + newsDate + "\n",
                        "\t -News Category: " + cat + "\n",
                        "\t -News url: " + aLink + "\n",
                        "\t -BreakPoint: get page" + "\n",
                        "\t -Exception Type: " + str(e.__class__) + "\n",
                        "\t -Exception Message: " + e.message + "\n",
                        "\t -Exception Traceback:", 
                        str(traceback.format_exc()) + "\n"
                    ]
                    f.writelines(ExList)
                    f.close
                    continue
                finally:
                    driver.quit()
                #########get page breakpoint

                #########content breakpoint
                try:
                    content=""
                    contentRaw = soupCont.select('.story > sectione > p')
                    try:
                        for i in xrange(0,len(contentRaw)):
                            for con in contentRaw[i].select('img'):
                                contentRaw[i].img.decompose()
                    except:
                        pass                    
                    for p in contentRaw:
                        content = content + p.text
                    content = content.strip()
                except Exception as e:
                    f=open("ETTodayException.log","a")
                    ExList=[
                        "Log at: " + str(datetime.today()) + "\n",
                        "\t -News Date: " + newsDate + "\n",
                        "\t -News Category: " + cat + "\n",
                        "\t -News url: " + aLink + "\n",
                        "\t -BreakPoint: content" + "\n",
                        "\t -Exception Type: " + str(e.__class__) + "\n",
                        "\t -Exception Message: " + e.message + "\n",
                        "\t -Exception Traceback:", 
                        str(traceback.format_exc()) + "\n"
                    ]
                    f.writelines(ExList)
                    f.close
                #########content breakpoint

                #########journist breakpoint
                jor = str(BeautifulSoup(cont).select('.story')[0].prettify().encode("utf-8"))
                try:
                    journalist=re.search('([^圖].*?)／.*?報導.*?', jor).group(1).strip()
                    journalist=journalist.translate(identify, delEStr)
                    if '記者' in journalist:
                        journalist=re.sub('記者','',journalist)
                    journalist=re.sub('、', '', journalist)
                except:
                    try:
                        journalist=re.search('文／(.*?)<',cont).group(1).strip()
                        journalist=journalist.translate(identify, delEStr)
                        journalist=re.sub('、', '', journalist)
                    except Exception as e:
                        journalist=None
                        f=open("ETTodayException.log","a")
                        ExList=[
                            "Log at: " + str(datetime.today()) + "\n",
                            "\t -News Date: " + newsDate + "\n",
                            "\t -News Category: " + cat + "\n",
                            "\t -News url: " + aLink + "\n",
                            "\t -BreakPoint: journist" + "\n",
                            "\t -Exception Type: " + str(e.__class__) + "\n",
                            "\t -Exception Message: " + e.message + "\n",
                            "\t -Exception Traceback: ", 
                            str(traceback.format_exc().decode('utf-8').encode('utf-8')) + "\n"
                        ]
                        f.writelines(ExList)
                        f.close
                    except:
                        print "jor: ",aLink," WTF"
                #########journist breakpoint

                #########category breakpoint
                category=block.select('em')[0].text
                '''
                if not category == cat.decode("utf-8"):
                    try:
                        raise Exception
                    except Exception as e:
                        f=open("ETTodayException.log","a")
                        ExList=[
                            "Log at: " + str(datetime.today()) + "\n",
                            "\t -News Date: " + newsDate + "\n",
                            "\t -News Category: " + cat + "\n",
                            "\t -News url: " + aLink + "\n",
                            "\t -BreakPoint: category" + "\n",
                            "\t -Exception Type: " + str(e.__class__) + "\n",
                            "\t -Exception Message: " + e.message + "\n",
                            "\t -Exception Traceback:", 
                            str(traceback.format_exc()) + "\n"
                        ]
                        f.writelines(ExList)
                        f.close
                '''
                #########category breakpoint

                try:
                    L.append({'class':category,'title':title,'newsDate':newsDate,'newsTime':newsTime,'journalist':journalist,'content':content,'hit':hit,'url':aLink,'source':'ETToday'})
                except:
                    pass

                #########comment breakpoint
                comments=[]
                try:
                    commLink0 = soupCont.find("div",re.compile('fb-comments.*'))
                    commLink1= commLink0.select(' span > iframe')[0]['src']
                    commLink=re.sub(r"numposts=\d{1,3}&", "numposts=100&", commLink1)
                    resComm = requests.get(commLink)
                    resComm.encoding = "utf-8"
                    soupComm = BeautifulSoup(resComm.text)      
                    commentBlock=soupComm.select('.postContainer.fsl.fwb.fcb')
                    for comm in commentBlock: 
                        dt=re.search("(\d\d\d\d).(\d{1,2}).(\d{1,2}). (\d{1,2}\:\d{1,2})",comm.select('abbr')[0]['title'])
                        commentDate="%s-%02d-%02d"%(dt.group(1),int(dt.group(2)),int(dt.group(3)))
                        commentTime=dt.group(4)
                        commenter=comm.select('.profileName')[0].text
                        commenterUrl=comm.select('.profileName')[0]['href']
                        comment=comm.select('.postText')[0].text
                        comments.append({'commentDate':commentDate,'commentTime':commentTime,'commenter':commenter,'commenterUrl':commenterUrl,'comment':comment,'url':aLink,'remark':None})
                except Exception as e:
                    f=open("ETTodayException.log","a")
                    ExList=[
                        "Log at: " + str(datetime.today()) + "\n",
                        "\t -News Date: " + newsDate + "\n",
                        "\t -News Category: " + cat + "\n",
                        "\t -News url: " + aLink + "\n",
                        "\t -BreakPoint: comment" + "\n",
                        "\t -Exception Type: " + str(e.__class__) + "\n",
                        "\t -Exception Message: " + e.message + "\n",
                        "\t -Exception Traceback: ", 
                        str(traceback.format_exc()) + "\n"
                    ]
                    f.writelines(ExList)
                    f.close
                #########comment breakpoint

                #################連資料庫用
                hit = None
                #print category,title,newsDate,newsTime,journalist,content,hit,aLink,commentDate,commentTime,commenter,comment
                try:
                    conn=connect("practice","oblivn0374","localhost","root")
                    insert_news(conn,category,title,newsDate,newsTime,journalist,content,hit,aLink)
                    insert_newsRaw(conn,aLink,res.text)
                    for c in comments:
                        insert_comments(conn,c.values()[3],c.values()[6],c.values()[4],c.values()[1],c.values()[0],c.values()[5])
                except:
                    print "something not write in DB \n", traceback.format_exc(),aLink,"\n"
                finally:
                    close(conn)
                #################連資料庫用
                time.sleep(sleep_time)
    
    
    #########write_in_csv breakpoint
    try:
        # pandas - content
        df = pd.DataFrame(L,columns =['class','title','newsDate','newsTime','journalist','content','hit','url','source'])
        # pandas - comment
        df_comm = pd.DataFrame(comments,columns =['commentDate','commentTime','commenter','commenterUrl','comment','url','remark'])
        
        # write csv
        if (str(year)+str(month))==(str(year2)+str(month2)):
            filename_content="ETTdoay_content_%d%02d.csv"%(year,month)
        else:
            filename_comment="ETTdoay_comment_%d%02d-%d%02d.csv"%(year,month,year2,month2)
        ## write content in csv
        f=open(filename_content,"w")
        df.to_csv(filename_content,encoding="utf-8")
        f.close()
        ## write comment in csv
        f=open(filename_comment,"w")
        df_comm.to_csv(filename_comment,encoding="utf-8")
        f.close()
        
    except Exception as e:
        f=open("ETTodayException.log","a")
        ExList=[
            "Log at: " + str(datetime.today()) + "\n",
            "\t -News Date: " + newsDate + "\n",
            "\t -News Category: " + cat + "\n",
            "\t -News url: " + aLink + "\n",
            "\t -BreakPoint: write_in_csv" + "\n",
            "\t -Exception Type: " + str(e.__class__) + "\n",
            "\t -Exception Message: " + e.message + "\n",
            "\t -Exception Traceback:", 
            str(traceback.format_exc()) + "\n"
        ]
        f.writelines(ExList)
        f.close
#########write_in_csv breakpoint

In [ ]:
# run
a = time.clock()
ETToday_month(2015,5,category=["政治","財經","國際","大陸"])
time.clock() - a

##直接輸入從今天算起要抓幾天 ETToday(days,category=["政治","財經","國際","大陸"],sleep_time=0)

In [ ]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import math
import re
from datetime import date,datetime, timedelta 
import time
import pandas as pd
from pandas.io import sql
import traceback
import string


#################連資料庫用
def connect(dbname,password,host,user,port=3306):
    import MySQLdb
    # 承德電腦 host="10.120.30.1"
    # localhost host = '127.0.0.1'
    db = MySQLdb.connect(host=host,user="root",passwd=password,db=dbname,charset="utf8")
    return db
def insert_news(db,category=None,title=None,newsDate=None,newsTime=None,journalist=None,content=None,hit=None,url=None,source="ETToday"):
    cur = db.cursor()
    cur.execute("insert into news(class,title,newsDate,newsTime,journalist,content,hit,url,source) values(%s,%s,%s,%s,%s,%s,%s,%s,%s)",[category,title,newsDate,newsTime,journalist,content,hit,url,source])
    db.commit()
def insert_comments(db,commentDate,commentTime,commenter,commenterUrl,comment,url,remark=None):
    cur = db.cursor()
    cur.execute("insert into comments(commentDate,commentTime,commenter,commenterUrl,comment,url)values(%s,%s,%s,%s,%s,%s)" ,[commentDate,commentTime,commenter,commenterUrl,comment,url])
    db.commit()
def insert_newsRaw(db,url,newsRaw):
    cur = db.cursor()
    cur.execute("insert into newsRaw(url,newsRaw)values(%s,%s)",[url,newsRaw])
    db.commit()
def close(db):
    db.close()
#################連資料庫用
    
def ETToday(days=1,sleep_time=0,category=["政治","財經","國際","大陸"]):
    
    ds = []
    for i in xrange(1,days+1):
        ds.append(str(date.today()- timedelta(i)))
    
    url_all =[]
    for newsDate in ds:
        url_dic = {"政治":"http://www.ettoday.net/news/news-list-%s-1.htm"%newsDate,
                    "財經":"http://www.ettoday.net/news/news-list-%s-17.htm"%newsDate,
                    "國際":"http://www.ettoday.net/news/news-list-%s-2.htm"%newsDate,
                    "大陸":"http://www.ettoday.net/news/news-list-%s-3.htm"%newsDate,
                    "社會":"http://www.ettoday.net/news/news-list-%s-6.htm"%newsDate,
                    "地方":"http://www.ettoday.net/news/news-list-%s-7.htm"%newsDate,
                    "影劇":"http://www.ettoday.net/news/news-list-%s-9.htm"%newsDate,
                    "3C":"http://www.ettoday.net/news/news-list-%s-20.htm"%newsDate
                   }
        url_categories = {}
        for cat in category:
            try:
                url_categories[str(cat)]=url_dic[str(cat)]
                url_all.append(url_categories[str(cat)])
            except:
                print newsDate, cat, url_dic[str(cat)], "\n"
                print traceback.format_exc() + "\n"

    identify = string.maketrans('', '')
    delEStr = string.punctuation
    L = []
    for cat in url_all:
        try:
            res = requests.get(cat)
            res.encoding="utf-8"
            soup = BeautifulSoup(res.text)
            page = re.search('\((\d+).\)',soup.select(".info")[0].text).group(1)
            pageCount = int(math.ceil(float(page)/30))
            soups = [soup]
        except:
            print cat
            raise 
        if pageCount >1:
            for i in xrange(2,pageCount+1):
                curl = str(cat).split(".htm")[0] + "-" + str(i) + ".htm"
                res = requests.get(curl)
                res.encoding="utf-8"
                soups.append(BeautifulSoup(res.text))

        for soup in soups:         
            for block in soup.select("#all-news-list > h3"):
                try:
                    timeRegex=re.search("\[(\d\d)\/(\d\d) (\d{1,2}\:\d{1,2})\]",block.select('span')[0].text)
                    testDate=timeRegex.group(1)+'-'+timeRegex.group(2)
                    if not testDate == re.search('\d{4}-(\d\d-\d\d)',newsDate).group(1).decode('utf-8'):
                        continue
                    newsTime=timeRegex.group(3)
                    title=block.select('a')[0].text
                    aLink=block.select('a')[0]['href']
                except:
                    print cat
                    raise
                #########get page breakpoint
                #driver = webdriver.Firefox()
                #driver = webdriver.Chrome()
                driver = webdriver.PhantomJS()
                webdriver.DesiredCapabilities.PHANTOMJS['phantomjs.page.customHeaders.Accept'] = 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'
                webdriver.DesiredCapabilities.PHANTOMJS['phantomjs.page.customHeaders.Accept-Encoding'] = 'gzip, deflate, sdch'
                webdriver.DesiredCapabilities.PHANTOMJS['phantomjs.page.customHeaders.Cache-Control'] = 'max-age=0'
                webdriver.DesiredCapabilities.PHANTOMJS['phantomjs.page.customHeaders.Connection'] = 'keep-alive'
                webdriver.DesiredCapabilities.PHANTOMJS['phantomjs.page.customHeaders.User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.124 Safari/537.36'
                try:    
                    driver.get(aLink)
                    WebDriverWait(driver, 30).until(
                            EC.visibility_of_element_located((By.ID, "fbComments"))
                        )
                    html = driver.page_source
                    cont = str(html.encode('utf-8'))
                    soupCont = BeautifulSoup(cont)
                except Exception as e:
                    f=open("ETTodayException.log","a")
                    ExList=[
                        "Log at: " + str(datetime.today()) + "\n",
                        "\t -News Date: " + newsDate + "\n",
                        "\t -News Category: " + cat + "\n",
                        "\t -News url: " + aLink + "\n",
                        "\t -BreakPoint: get page" + "\n",
                        "\t -Exception Type: " + str(e.__class__) + "\n",
                        "\t -Exception Message: " + e.message + "\n",
                        "\t -Exception Traceback:", 
                        str(traceback.format_exc()) + "\n"
                    ]
                    f.writelines(ExList)
                    f.close
                    continue
                finally:
                    driver.quit()
                #########get page breakpoint

                #########content breakpoint
                try:
                    content=""
                    contentRaw = soupCont.select('.story > sectione > p')
                    try:
                        for i in xrange(0,len(contentRaw)):
                            for con in contentRaw[i].select('img'):
                                contentRaw[i].img.decompose()
                    except:
                        pass
                        for p in contentRaw:
                            content = content + p.text
                        content = content.strip()
                except Exception as e:
                    f=open("ETTodayException.log","a")
                    ExList=[
                        "Log at: " + str(datetime.today()) + "\n",
                        "\t -News Date: " + newsDate + "\n",
                        "\t -News Category: " + cat + "\n",
                        "\t -News url: " + aLink + "\n",
                        "\t -BreakPoint: content" + "\n",
                        "\t -Exception Type: " + str(e.__class__) + "\n",
                        "\t -Exception Message: " + e.message + "\n",
                        "\t -Exception Traceback:", 
                        str(traceback.format_exc()) + "\n"
                    ]
                    f.writelines(ExList)
                    f.close
                #########content breakpoint

                #########journist breakpoint
                jor = str(BeautifulSoup(cont).select('.story')[0].prettify().encode("utf-8"))
                try:
                    journalist=re.search('([^圖].*?)／.*?報導.*?',jor).group(1).strip()
                    journalist=journalist.translate(identify, delEStr)
                    if '記者' in journalist:
                        journalist=re.sub('記者','',journalist)
                    journalist=re.sub('、', '', journalist)
                except:
                    try:
                        journalist=re.search('文／(.*?)<',cont).group(1).strip()
                        journalist=journalist.translate(identify, delEStr)
                        journalist=re.sub('、', '', journalist)
                    except Exception as e:
                        journalist=None
                        f=open("ETTodayException.log","a")
                        ExList=[
                            "Log at: " + str(datetime.today()) + "\n",
                            "\t -News Date: " + newsDate + "\n",
                            "\t -News Category: " + cat + "\n",
                            "\t -News url: " + aLink + "\n",
                            "\t -BreakPoint: journist" + "\n",
                            "\t -Exception Type: " + str(e.__class__) + "\n",
                            "\t -Exception Message: " + e.message + "\n",
                            "\t -Exception Traceback: ", 
                            str(traceback.format_exc().decode('utf-8').encode('utf-8')) + "\n"
                        ]
                        f.writelines(ExList)
                        f.close
                    except:
                        print "jor: ",aLink," WTF"
                #########journist breakpoint

                #########category breakpoint
                category=block.select('em')[0].text
                '''if not category == cat.decode("utf-8"):
                    try:
                        raise Exception
                    except Exception as e:
                        f=open("ETTodayException.log","a")
                        ExList=[
                            "Log at: " + str(datetime.today()) + "\n",
                            "\t -News Date: " + newsDate + "\n",
                            "\t -News Category: " + cat + "\n",
                            "\t -News url: " + aLink + "\n",
                            "\t -BreakPoint: category" + "\n",
                            "\t -Exception Type: " + str(e.__class__) + "\n",
                            "\t -Exception Message: " + e.message + "\n",
                            "\t -Exception Traceback:", 
                            str(traceback.format_exc()) + "\n"
                        ]
                        f.writelines(ExList)
                        f.close'''
                #########category breakpoint

                try:
                    L.append({'class':category,'title':title,'newsDate':newsDate,'newsTime':newsTime,'journalist':journalist,'content':content,'hit':hit,'url':aLink,'source':'ETToday'})
                except:
                    pass

                #########comment breakpoint
                comments=[]
                try:
                    commLink0 = soupCont.find("div",re.compile('fb-comments.*'))
                    commLink1= commLink0.select(' span > iframe')[0]['src']
                    commLink=re.sub(r"numposts=\d{1,3}&", "numposts=100&", commLink1)
                    resComm = requests.get(commLink)
                    resComm.encoding = "utf-8"
                    soupComm = BeautifulSoup(resComm.text)      
                    commentBlock=soupComm.select('.postContainer.fsl.fwb.fcb')
                    for comm in commentBlock: 
                        dt=re.search("(\d\d\d\d).(\d{1,2}).(\d{1,2}). (\d{1,2}\:\d{1,2})",comm.select('abbr')[0]['title'])
                        commentDate="%s-%02d-%02d"%(dt.group(1),int(dt.group(2)),int(dt.group(3)))
                        commentTime=dt.group(4)
                        commenter=comm.select('.profileName')[0].text
                        commenterUrl=comm.select('.profileName')[0]['href']
                        comment=comm.select('.postText')[0].text
                        comments.append({'commentDate':commentDate,'commentTime':commentTime,'commenter':commenter,'commenterUrl':commenterUrl,'comment':comment,'url':aLink,'remark':None})
                except Exception as e:
                    f=open("ETTodayException.log","a")
                    ExList=[
                        "Log at: " + str(datetime.today()) + "\n",
                        "\t -News Date: " + newsDate + "\n",
                        "\t -News Category: " + cat + "\n",
                        "\t -News url: " + aLink + "\n",
                        "\t -BreakPoint: comment" + "\n",
                        "\t -Exception Type: " + str(e.__class__) + "\n",
                        "\t -Exception Message: " + e.message + "\n",
                        "\t -Exception Traceback: ", 
                        str(traceback.format_exc()) + "\n"
                    ]
                    f.writelines(ExList)
                    f.close
                #########comment breakpoint

                #################連資料庫用
                hit = None
                #print category,title,newsDate,newsTime,journalist,content,hit,aLink,commentDate,commentTime,commenter,comment
                try:
                    conn=connect("practice","oblivn0374","localhost","root")
                    insert_news(conn,category,title,newsDate,newsTime,journalist,content,hit,aLink)
                    insert_newsRaw(conn,aLink,res.text)
                    for c in comments:
                        insert_comments(conn,c.values()[3],c.values()[6],c.values()[4],c.values()[1],c.values()[0],c.values()[5])
                except:
                    print "something not write in DB \n", traceback.format_exc(),aLink
                finally:
                    close(conn)
                #################連資料庫用
                time.sleep(sleep_time)
    
    #########write_in_csv breakpoint
    try:
        # pandas - content
        df = pd.DataFrame(L,columns =['class','title','newsDate','newsTime','journalist','content','hit','url','source'])
        # pandas - comment
        df_comm = pd.DataFrame(comments,columns =['commentDate','commentTime','commenter','commenterUrl','comment','url','remark'])
                               
        # write content in csv
        f=open("ETToday_content.csv","w")
        df.to_csv("ETToday_content.csv",encoding="utf-8")
        f.close()
        # write comment in csv
        f=open("ETToday_comment.csv","w")
        df_comm.to_csv("ETToday_comment.csv",encoding="utf-8")
        f.close()
        
    except Exception as e:
        f=open("ETTodayException.log","a")
        ExList=[
            "Log at: " + str(datetime.today()) + "\n",
            "\t -News Date: " + newsDate + "\n",
            "\t -News Category: " + cat + "\n",
            "\t -News url: " + aLink + "\n",
            "\t -BreakPoint: write_in_csv" + "\n",
            "\t -Exception Type: " + str(e.__class__) + "\n",
            "\t -Exception Message: " + e.message + "\n",
            "\t -Exception Traceback:", 
            str(traceback.format_exc()) + "\n"
        ]
        f.writelines(ExList)
        f.close
#########write_in_csv breakpoint 

In [ ]:
# run
a = time.clock()
ETToday()
time.clock() - a

## For Test

In [108]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import math
import re
from datetime import date,datetime, timedelta 
import time
import pandas as pd
from pandas.io import sql
import traceback
import string
driver = webdriver.PhantomJS()
webdriver.DesiredCapabilities.PHANTOMJS['phantomjs.page.customHeaders.Accept'] = 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'
webdriver.DesiredCapabilities.PHANTOMJS['phantomjs.page.customHeaders.Accept-Encoding'] = 'gzip, deflate, sdch'
webdriver.DesiredCapabilities.PHANTOMJS['phantomjs.page.customHeaders.Cache-Control'] = 'max-age=0'
webdriver.DesiredCapabilities.PHANTOMJS['phantomjs.page.customHeaders.Connection'] = 'keep-alive'
webdriver.DesiredCapabilities.PHANTOMJS['phantomjs.page.customHeaders.User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.124 Safari/537.36'
aLink="http://www.ettoday.net/news/20150620/523337.htm"
try:    
    driver.get(aLink)
    WebDriverWait(driver, 30).until(
            EC.visibility_of_element_located((By.ID, "fbComments"))
        )
    html = driver.page_source
    cont = str(html.encode('utf-8'))
    soupCont = BeautifulSoup(cont)
except Exception as e:
    f=open("ETTodayException.log","a")
    ExList=[
        "Log at: " + str(datetime.today()) + "\n",
        "\t -News Date: " + newsDate + "\n",
        "\t -News Category: " + cat + "\n",
        "\t -News url: " + aLink + "\n",
        "\t -BreakPoint: get page" + "\n",
        "\t -Exception Type: " + str(e.__class__) + "\n",
        "\t -Exception Message: " + e.message + "\n",
        "\t -Exception Traceback:", 
        traceback.format_exc() + "\n"
    ]
    f.writelines(ExList)
    f.close
finally:
    driver.quit()

In [ ]:
comments=[]
commLink0 = soupCont.find("div",re.compile('fb-comments.*'))
commLink1= commLink0.select(' span > iframe')[0]['src']
commLink=re.sub(r"numposts=\d{1,3}&", "numposts=100&", commLink1)
resComm = requests.get(commLink)
resComm.encoding = "utf-8"
soupComm = BeautifulSoup(resComm.text)      
commentBlock=soupComm.select('.postContainer.fsl.fwb.fcb')
for comm in commentBlock: 
    dt=re.search("(\d\d\d\d).(\d{1,2}).(\d{1,2}). (\d{1,2}\:\d{1,2})",comm.select('abbr')[0]['title'])
    commentDate="%s-%02d-%02d"%(dt.group(1),int(dt.group(2)),int(dt.group(3)))
    commentTime=dt.group(4)
    commenter=comm.select('.profileName')[0].text
    commenterUrl=comm.select('.profileName')[0]['href']
    comment=comm.select('.postText')[0].text
    comments.append({'commentDate':commentDate,'commentTime':commentTime,'commenter':commenter,'commenterUrl':commenterUrl,'comment':comment,'url':aLink,'remark':None})

conn=connect("practice","oblivn0374","localhost","root")
for c in comments:
    print c.values()[3],c.values()[6],c.values()[4],c.values()[1],c.values()[0],c.values()[5]
    insert_comments(conn,c.values()[3],c.values()[6],c.values()[4],c.values()[1],c.values()[0],c.values()[5])
close(conn)

In [4]:
webdriver.DesiredCapabilities.PHANTOMJS['phantomjs.page.customHeaders.Accept'] = 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'
webdriver.DesiredCapabilities.PHANTOMJS['phantomjs.page.customHeaders.Accept-Encoding'] = 'gzip, deflate, sdch'
webdriver.DesiredCapabilities.PHANTOMJS['phantomjs.page.customHeaders.Cache-Control'] = 'max-age=0'
webdriver.DesiredCapabilities.PHANTOMJS['phantomjs.page.customHeaders.Connection'] = 'keep-alive'
webdriver.DesiredCapabilities.PHANTOMJS['phantomjs.page.customHeaders.User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.124 Safari/537.36'

In [ ]:
import requests
url="https://www.facebook.com/yijuamy.chen"
headers={
':host':'www.facebook.com',
':method':'GET',
':path':'/yijuamy.chen',
':scheme':'https',
':version':'HTTP/1.1',
'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
'accept-encoding':'gzip, deflate, sdch',
'accept-language':'en-US,en;q=0.8,zh-CN;q=0.6,zh;q=0.4',
'cache-control':'max-age=0',
'cookie':'datr=QBNeUwqI4S51V5AzGEoeCA1r; _ga=GA1.2.1275457108.1428803544; lu=TgMfpfq2tjUWZNCGMvAtPqgg; c_user=100000310235880; fr=0R1TtmugtGGZqdare.AWXzmmSln6UThlEI7X7DWth-8LI.BT6iv-.MD.FV-.0.AWXKXYOq; xs=46%3AcvZCripsh73rfw%3A2%3A1434196224%3A14635; csm=2; s=Aa5VbbutJxLdjkjE.BVfBkA; act=1434791149057%2F13; p=-2; presence=EM434791329EuserFA21B00310235880A2EstateFDsb2F0Et2F_5b_5dElm2FnullEuct2F1434788575BEtrFA2loadA2EtwF2521827791EatF1434791328792G434791159662CEchFDp_5f1B00310235880F0CC; wd=1239x259',
'referer':'https://www.facebook.com/plugins/comments.php?api_key=146858218737386&channel_url=http%3A%2F%2Fstatic.ak.facebook.com%2Fconnect%2Fxd_arbiter%2F1ldYU13brY_.js%3Fversion%3D41%23cb%3Df3be262b0c%26domain%3Dwww.ettoday.net%26origin%3Dhttp%253A%252F%252Fwww.ettoday.net%252Ffc8173f04%26relation%3Dparent.parent&colorscheme=light&href=http%3A%2F%2Fwww.ettoday.net%2Fnews%2F20150612%2F520021.htm&locale=zh_TW&numposts=10&sdk=joey&skin=light&version=v2.0&width=620',
'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.124 Safari/537.36'
}

